In [687]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


read from a file named “input.txt” containing two sequences of 16 comma-
delimited integers in the first line. The second sequence should be delimited by
2
a semicolon (e.g., 1, 2, 3, 4, 5, . . . , 16; 1, 2, 3, 4, 5, . . . , 16). 

The following functions are given in the project description

In [688]:
def algorithm_1(matrix_a: np.ndarray, matrix_b: np.ndarray, matrix_c: np.ndarray, 
                p: int, q: int, r: int) -> None:
    """
    Multiply two matrices using the traditional algorithm
    Parameters
    ----------
    matrix_a : np.ndarray
        The first matrix to multiply
    matrix_b : np.ndarray
        The second matrix to multiply
    matrix_c : np.ndarray
        The matrix to store the result in
    p : int
        The number of rows in matrix_a
    q : int
        The number of columns in matrix_a and the number of rows in matrix_b
    r : int
        The number of columns in matrix_b
    Returns
    -------
    None
    """
    for i in range(p):
        for j in range(r):
            for k in range(q):
                matrix_c[i][j] += matrix_a[i][k] * matrix_b[k][j]
    return

In [689]:
def algorithm_2(matrix_a: np.ndarray, matrix_b: np.ndarray, matrix_c: np.ndarray, p: int, 
                q: int, r: int, T: int =5) -> None:
    """
    Multiply two matricies using block matrix multiplication
    Parameters
    ----------
    matrix_a : np.ndarray
        The first matrix to multiply
    matrix_b : np.ndarray
        The second matrix to multiply
    matrix_c : np.ndarray
        The matrix to store the result in
    p : int
        The number of rows in matrix_a
    q : int
        The number of columns in matrix_a and the number of rows in matrix_b
    r : int
        The number of columns in matrix_b
    T : int
        The size of the tiles
    Returns
    -------
    None
    """
    for i in range(0, p, T):
        for j in range(0, r, T):
            for k in range(0, q, T):
                for i1 in range(i, min(i + T, p)):
                    for j1 in range(j, min(j + T, r)):
                        for k1 in range(k, min(k + T, q)):
                            matrix_c[i1][j1] += matrix_a[i1][k1] * matrix_b[k1][j1]
    return None

In [690]:
def algorithm_3(matrix_a: np.ndarray, matrix_b: np.ndarray, matrix_c: np.ndarray, p: int, q: int, r: int) -> None:
    """
    split the matricies into 2 submatricies and multiply them using the traditional algorithm
    Parameters
    ----------
    matrix_a : np.ndarray
        The first matrix to multiply
    matrix_b : np.ndarray
        The second matrix to multiply
    matrix_c : np.ndarray
        The matrix to store the result in
    p : int
        The number of rows in matrix_a
    q : int
        The number of columns in matrix_a and the number of rows in matrix_b
    r : int
        The number of columns in matrix_b
    Returns
    -------
    None
    """
    if max(p, q, r) <= 2:
        algorithm_1(matrix_a, matrix_b, matrix_c, p, q, r)
    else:
        if max(p, q, r) == p:
            # split A horizontally
            p1 = p // 2
            algorithm_3(matrix_a[:p1, :], matrix_b, matrix_c[:p1, :], p1, q, r)
            algorithm_3(matrix_a[p1:, :], matrix_b, matrix_c[p1:, :], p - p1, q, r)
        elif max(p, q, r) == q:
            q1 = q // 2
            # split A vertically and B horizontally
            algorithm_3(matrix_a[:, :q1], matrix_b[:q1, :], matrix_c, p, q1, r)
            algorithm_3(matrix_a[:, q1:], matrix_b[q1:, :], matrix_c, p, q - q1, r)
        else:
            r1 = r // 2
            # split B vertically
            algorithm_3(matrix_a, matrix_b[:, :r1 + 1], matrix_c[:, :r1 + 1], p, q, r1)
            algorithm_3(matrix_a, matrix_b[:, r1:], matrix_c[:, r1:], p, q, r - r1)
    return None

In [691]:
def algorithm_4(matrix_a: np.ndarray, matrix_b: np.ndarray, matrix_c: np.ndarray) -> np.ndarray:
    """
    Block matrix multiplication with a recursive call using 4 blocks
    Parameters
    ----------
    matrix_a : np.ndarray
        The first matrix to multiply
    matrix_b : np.ndarray
        The second matrix to multiply
    matrix_c : np.ndarray
        The matrix to store the result in
    Returns
    -------
    False if the matrices are not the same size
    """

    p, q = matrix_a.shape
    r, s = matrix_b.shape
    if p != q != r != s:
        raise ValueError('Matrices are not the same size or not square matrices.')
    if p == 1:
        val = matrix_a[0][0] * matrix_b[0][0]
        return val
    else:
        p1 = p // 2
        q1 = q // 2
        s1 = s // 2

        # split the matrices into 4 submatrices
        a11 = matrix_a[:p1, :q1]
        a12 = matrix_a[:p1, q1:]
        a21 = matrix_a[p1:, :q1]
        a22 = matrix_a[p1:, q1:]

        b11 = matrix_b[:p1, :q1]
        b12 = matrix_b[:p1, q1:]
        b21 = matrix_b[p1:, :q1]
        b22 = matrix_b[p1:, q1:]
        
        c11 = matrix_c[:p1, :s1]
        c12 = matrix_c[:p1, s1:]
        c21 = matrix_c[p1:, :s1]
        c22 = matrix_c[p1:, s1:]

        # we use the recursive call to calculate the submatrices
        # the temp_c variables are used to store the results of the recursive calls
        # we cant just use the normal matrix_c because we are using the same matrix_c to store the results of the recursive calls
        # and we do not want to overwrite the results of the recursive calls
        temp_c11 = algorithm_4(a11, b11, np.zeros([p1,q1])) + algorithm_4(a12, b21, np.zeros([p1,q1]))
        c11 = temp_c11
        temp_c12 = algorithm_4(a11, b12, np.zeros([p1,q1])) + algorithm_4(a12, b22, np.zeros([p1,q1]))
        c12 = temp_c12
        temp_c21 = algorithm_4(a21, b11, np.zeros([p1,q1])) + algorithm_4(a22, b21, np.zeros([p1,q1]))
        c21 = temp_c21
        temp_c22 = algorithm_4(a21, b12, np.zeros([p1,q1])) + algorithm_4(a22, b22, np.zeros([p1,q1]))
        c22 = temp_c22

        # Combine submatrices to form the resulting matrix C
        matrix_c[:p1, :s1] = c11
        matrix_c[:p1, s1:] = c12
        matrix_c[p1:, :s1] = c21
        matrix_c[p1:, s1:] = c22


        return matrix_c

In [692]:
def algorithm_5(matrix_a: np.ndarray, matrix_b: np.ndarray, matrix_c: np.ndarray) -> None:
    """
    Strassen's algorithm for matrix multiplication
    https://www.youtube.com/watch?app=desktop&v=0oJyNmEbS4w
    Parameters
    ----------
    matrix_a : np.ndarray
        The first matrix to multiply
    matrix_b : np.ndarray
        The second matrix to multiply
    matrix_c : np.ndarray
        The matrix to store the result in
    Returns
    -------
    None
    """
    p, q = matrix_a.shape
    r, s = matrix_b.shape
    if p != q != r != s or p != r or q != s:
        raise ValueError('Matrices are not the same size or not square matrices.')
    if p < 3:
        algorithm_1(matrix_a, matrix_b, matrix_c, p, q, r)
    else:
        # split the matrices into 4 submatrices
        p1 = p // 2
        q1 = q // 2
        r1 = r // 2
        s1 = s // 2
        
        a11 = matrix_a[:p1, :q1]
        a12 = matrix_a[:p1, q1:]
        a21 = matrix_a[p1:, :q1]
        a22 = matrix_a[p1:, q1:]
        b11 = matrix_b[:q1, :s1]
        b12 = matrix_b[:q1, s1:]
        b21 = matrix_b[q1:, :s1]
        b22 = matrix_b[q1:, s1:]

        # recursively calculate the 7 parts 
        m1 = algorithm_5(np.add(a11, a22), np.add(b11, b22), np.zeros([p1, s1]))
        m2 = algorithm_5(np.add(a21, a22), b11, np.zeros([p1, s1]))
        m3 = algorithm_5(a11, np.subtract(b12, b22), np.zeros([p1, s1]))
        m4 = algorithm_5(a22, np.subtract(b21, b11), np.zeros([p1, s1]))
        m5 = algorithm_5(np.add(a11, a12), b22, np.zeros([p1, s1]))
        m6 = algorithm_5(np.subtract(a21, a11), np.add(b11, b12), np.zeros([p1, s1]))
        m7 = algorithm_5(np.subtract(a12, a22), np.add(b21, b22), np.zeros([p1, s1]))

        # calculate the submatrices
        c11 = np.subtract(np.add(np.add(m1, m4), m7), m5)
        c12 = np.add(m3, m5)
        c21 = np.add(m2, m4)
        c22 = np.subtract(np.add(np.add(m1, m3), m6), m2)

        # Combine submatrices to form the resulting matrix C
        matrix_c[:p1, :s1] = c11
        matrix_c[:p1, s1:] = c12
        matrix_c[p1:, :s1] = c21
        matrix_c[p1:, s1:] = c22

    return matrix_c




In [693]:
def matrix_file_in_all_algos(file_name: str) -> bool:
    """
    Read a file with two 4x4 matrices and run each of the 5 algorithms
    Parameters
    ----------
    file_name : str
        The name of the file to read
    Returns
    -------
    bool
        True if the file was read successfully, False otherwise
    """
    try:
        with open(file_name, 'r') as file:
            # read file into a list
            lines = file.readlines()
            lines = lines[0].split(';')
            lines = [line.split(',') for line in lines]
            lines = [[int(x) for x in line] for line in lines]
            

            
            # convert list into a numpy array
            matrix_a = np.array(lines[0]).reshape(4, 4)
            matrix_b = np.array(lines[1]).reshape(4, 4)
            matrix_c = np.zeros([4, 4])

            # run each algorithm
            algorithm_1(matrix_a, matrix_b, matrix_c, 4, 4, 4)
            print("Algorithm 1:\n", matrix_c)
            matrix_c = np.zeros([4, 4])

            algorithm_2(matrix_a, matrix_b, matrix_c, 4, 4, 4)
            print("\n\nAlgorithm 2:\n", matrix_c)
            matrix_c = np.zeros([4, 4])

            algorithm_3(matrix_a, matrix_b, matrix_c, 4, 4, 4)
            print("\n\nAlgorithm 3:\n", matrix_c)
            matrix_c = np.zeros([4, 4])

            matrix_c = algorithm_4(matrix_a, matrix_b, matrix_c) # this algorithm is not working
            print("\n\nAlgorithm 4:\n", matrix_c)
            matrix_c = np.zeros([4, 4])

            matrix_c = algorithm_5(matrix_a, matrix_b, matrix_c) # this algorithm runs but the result is not correct
            print("\n\nAlgorithm 5:\n", matrix_c)
            matrix_c = np.zeros([4, 4])


            return
    except FileNotFoundError:

        return False
    
matrix_file_in_all_algos('input.txt')
    

Algorithm 1:
 [[ 189.  100.  110.  120.]
 [ 697.  228.  254.  280.]
 [1205.  356.  398.  440.]
 [1713.  484.  542.  600.]]


Algorithm 2:
 [[ 189.  100.  110.  120.]
 [ 697.  228.  254.  280.]
 [1205.  356.  398.  440.]
 [1713.  484.  542.  600.]]


Algorithm 3:
 [[ 189.  100.  110.  120.]
 [ 697.  228.  254.  280.]
 [1205.  356.  398.  440.]
 [1713.  484.  542.  600.]]


Algorithm 4:
 [[ 189.  100.  110.  120.]
 [ 697.  228.  254.  280.]
 [1205.  356.  398.  440.]
 [1713.  484.  542.  600.]]


Algorithm 5:
 [[ 189.  100.  110.  120.]
 [ 697.  228.  254.  280.]
 [1205.  356.  398.  440.]
 [1713.  484.  542.  600.]]
